In [0]:
### Import libraries 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
# import scipy 
import re
import os
import sys 
# from datetime import datetime
!pip install -q fuzzywuzzy
!pip install -q fuzzywuzzy[speedup]
from fuzzywuzzy import fuzz

%matplotlib inline 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
SHARED_PATH = "drive/Shared drives/1MR - BigData/Project 2020/City Transaction Data/OUTPUT/"
STANDARD_DATA = SHARED_PATH+"Novi_Sub_Pub-All.xlsx"
JAMESTOWN_PATH = "drive/Shared drives/1MR - BigData/Project 2020/Subdivisions/Novi/Jamestown/Novi_Transaction.xlsx"
MY_PATH = "drive/My Drive/temp/"
PARSER_PATH = "drive/My Drive/address_parser/"

In [0]:
sys.path.append(PARSER_PATH)
import address_parser.address_methods as amod
from address_parser_class import AddressParser

In [0]:
ap = AddressParser()

# Clean dataset

In [0]:
df_novi = pd.read_excel(JAMESTOWN_PATH, 
                      usecols=['Address_x','Subdivision Name','Parcel Number'])\
                      .rename(columns={'Parcel Number':'PIN'})

In [0]:
df_test = amod.gen_raw_index(df_novi)
df_test = amod.gen_PIN_len(df_test)
df_test = amod.gen_mult_PIN_list(df_test)
df_test = amod.split_PIN_list(df_test)
df_test = amod.clean_temp_col(df_test)
df_test = amod.gen_index(df_test)

In [0]:
len(df_test)

15261

# Merge with public record

In [0]:
def load_public_record(path, starting_pattern, full_record=False):
    dir_prd = os.fsencode(path)
    public_record = pd.DataFrame()
    for f in os.listdir(dir_prd):
        if (not starting_pattern) or (os.fsdecode(f).startswith(starting_pattern)): 
            # print(os.fsdecode(f))
            if full_record:
                new_record = pd.read_csv(path+os.fsdecode(f)).rename(\
                                         columns={'Property ID':'PIN', 
                                                  'Property Address':'Address'})
            else:
                new_record = pd.read_csv(path+os.fsdecode(f), usecols=[0,1,15], 
                                     skiprows=1, 
                                     names=['PIN','Address','Subdivision Name'])
            public_record = public_record.append(new_record, ignore_index=True)
            # print(len(public_record))
    return public_record.drop_duplicates()
    
def merge_public_record(df, public_record): 
    return df.astype({'PIN':str}).merge(public_record.astype({'PIN':str}),
                                             on='PIN', how='inner')
def gen_not_matched(df_merged, df_all):
    return df_all.loc[~(df_all['index_clean'].isin(
        df_merged['index_clean']
    ))]


In [0]:
# Load public records 
path = "/content/drive/Shared drives/1MR - BigData/Project 2020/City Transaction Data/OUTPUT/"
public_record = load_public_record(path, 'NoviPRD', True)

In [0]:
# Merge transaction with public record
df_merged = merge_public_record(df_test, public_record)

In [0]:
# Separate not matched records 
df_not_matched = gen_not_matched(df_merged, df_test)

# Analyze not matched

In [0]:
# Prepare datasets to analyze in details
# i.e., split 'Address' in parts like "street name", "street suffix" etc 
split_address = amod.parse_street_series(df_not_matched['Address_x'], df_not_matched)
prd_split = amod.parse_street_series(public_record['Address'], public_record[['PIN', 'Address']])

In [0]:
# Summarize information 
name_to_suffix_dict = amod.get_combinations_as_dict(prd_split, ['Street Name', 'Street Suffix'])
std_name_set = set(prd_split['Street Name'].unique())
std_address_set = set( zip(prd_split['Street Name'], prd_split['Street Suffix']) )

In [0]:
def close_match_row(row): 
    name_sug, suffix_sug, score = ap.close_match(row['Street Name'], 
                                              row['Street Suffix'], std_address_set, std_name_set, 
                          name_to_suffix_dict)        
    return (name_sug, suffix_sug, score, row['index'])

def agg_func(x):
    ans = set(x)
    if len(ans)==1:
        return list(ans)[0]
    return ans 

In [0]:
# Create empty dataset to store matches based on address 
temp = pd.DataFrame(columns=['Name_sug', 'Suffix_sug', 'Score', 'index'])

In [0]:
# Get suggested name and suffix 
temp['Name_sug'], temp['Suffix_sug'], temp['Score'], temp['index'] = \
    zip(*split_address.reset_index().apply(close_match_row, axis=1))

In [0]:
# Incorporate information from split_address
temp = split_address.reset_index().merge(temp, on='index', how='inner')

In [0]:
# If match score >= 90, replace old 'Street Name' and 'Street Suffix' with 
#     suggested ones. 
temp.loc[temp.Score>=90, ['Street Name', 'Street Suffix']] = np.nan, np.nan
temp['Street Name'] = temp['Street Name'].fillna(temp['Name_sug'])
temp['Street Suffix'] = temp['Street Suffix'].fillna(temp['Suffix_sug'])

In [0]:
# Merge with public record to get standard PIN 
temp = temp.merge(prd_split, on=['Street Number', 'Street Name', 'Street Suffix'], how='inner')

In [0]:
# There might be multiple matching address (different units in the same address)
# We group by index to make the match one-to-many (i.e. multiple matches with 
#   the same address should appear in the same row)
g = temp[['index', 'Address_x', 'PIN_x', 'Address', 'PIN_y', 'index_clean', 
          'index_raw']].groupby('index')
temp = pd.concat([g.agg(agg_func), g.size()], axis=1).rename(columns={0:'num_match'})

In [0]:
# Get the ones with unique match. These can be appended to df_merged 
temp1 = temp.rename(columns={'PIN_x':'PIN_from_transaction', 'PIN_y':'PIN'})\
    .loc[temp.num_match==1].drop(columns=['num_match'])
temp1['Subdivision Name'] = np.nan
df_merged1 = merge_public_record(temp1[(df_test.columns)], public_record)
df_merged = df_merged.append(df_merged1, ignore_index=True, sort=False)

In [0]:
# Re-calculate the records that are not maatched 
df_not_matched = gen_not_matched(df_merged, df_test)

In [0]:
# Check that the original file has indeed been split into 2 disjoint sets 
print(len(df_not_matched), len(df_merged), len(df_test))

128 15133 15261


In [0]:
# Save the results to your dir
saving_dir = "/content/drive/My Drive/test_merge/"
gen_not_matched(df_merged, temp).to_csv(saving_dir + 'matched_on_address.csv')
df_merged.to_csv(saving_dir + 'merged.csv')
df_not_matched.to_csv(saving_dir + 'not_matched.csv')